# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 3**

**Fecha**: 11 febrero 2025

**Nombre del Estudiante**: Tirzah Peniche Barba

**Profesor**: Pablo Camarillo Ramirez

ENCONTRAR LA INSTALACIÓN DE PYSPARK

In [2]:
import findspark
findspark.init()

CREACIÓN DE LA CONEXIÓN CON EL CLUSTER DE SPARK

In [3]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("Map-Reduce-Examples") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/11 15:01:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


CREAR EL RDD

In [13]:
posts = ["Learning #BigData with #PySpark is fun! #AI",
"#AI is transforming the world. #BigData #MachineLearning",
"I love #PySpark and #BigData. #AI #DataScience",
"#DataScience and #AI are the future. #BigData",
"#PySpark is awesome! #BigData #AI"]

post_rdd = sc.parallelize(posts)

EXTRAER LOS HASHTAGS

In [14]:
def get_hashtags(post):
    return [word for word in post.split() if word.startswith('#')]

hashtags_rdd = post_rdd.flatMap(get_hashtags)
hashtags_rdd.collect()

['#BigData',
 '#PySpark',
 '#AI',
 '#AI',
 '#BigData',
 '#MachineLearning',
 '#PySpark',
 '#BigData.',
 '#AI',
 '#DataScience',
 '#DataScience',
 '#AI',
 '#BigData',
 '#PySpark',
 '#BigData',
 '#AI']

CREAR UN RDD CON LAS TUPLAS (HASHTAG, 1)

In [15]:
tuples_hashtags_rdd = hashtags_rdd.map(lambda x: (x, 1))
tuples_hashtags_rdd.collect()

[('#BigData', 1),
 ('#PySpark', 1),
 ('#AI', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#MachineLearning', 1),
 ('#PySpark', 1),
 ('#BigData.', 1),
 ('#AI', 1),
 ('#DataScience', 1),
 ('#DataScience', 1),
 ('#AI', 1),
 ('#BigData', 1),
 ('#PySpark', 1),
 ('#BigData', 1),
 ('#AI', 1)]

CONTAR OCURRENCIAS DE HASHTAGS

In [16]:
hashtags_count = hashtags_rdd.countByValue()
for word, count in hashtags_count.items():
    print(f"{word}: {count}")

#BigData: 4
#PySpark: 3
#AI: 5
#MachineLearning: 1
#BigData.: 1
#DataScience: 2


CREAR TUPLAS DE HASHTAGS POR LONGITUD

In [19]:
tuples_length_hashtags_rdd = hashtags_rdd.map(lambda x: (len(x), x))
tuples_length_hashtags_rdd.collect()

[(8, '#BigData'),
 (8, '#PySpark'),
 (3, '#AI'),
 (3, '#AI'),
 (8, '#BigData'),
 (16, '#MachineLearning'),
 (8, '#PySpark'),
 (9, '#BigData.'),
 (3, '#AI'),
 (12, '#DataScience'),
 (12, '#DataScience'),
 (3, '#AI'),
 (8, '#BigData'),
 (8, '#PySpark'),
 (8, '#BigData'),
 (3, '#AI')]

AGRUPAR HASHTAGS POR LONGITUD

In [27]:
grouped_rdd = tuples_length_hashtags_rdd.groupByKey()
result = grouped_rdd.collect()
for key, values in result:
    print(f"Length {key}: {list(values)}")

Length 8: ['#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData', '#PySpark', '#BigData']
Length 16: ['#MachineLearning']
Length 12: ['#DataScience', '#DataScience']
Length 3: ['#AI', '#AI', '#AI', '#AI', '#AI']
Length 9: ['#BigData.']


SEPARAR LOS HASHTAGS DIFERENTES CON LA MISMA LONGITUD

In [35]:
grouped_rdd = tuples_length_hashtags_rdd.map(lambda x: ((x[0], x[1]), x[1])).groupByKey()
result = grouped_rdd.mapValues(list).collect()
for (key, hashtag), values in result:
    print(f"Length {key}: {values}")


Length 8: ['#BigData', '#BigData', '#BigData', '#BigData']
Length 12: ['#DataScience', '#DataScience']
Length 8: ['#PySpark', '#PySpark', '#PySpark']
Length 3: ['#AI', '#AI', '#AI', '#AI', '#AI']
Length 16: ['#MachineLearning']
Length 9: ['#BigData.']


In [ ]:
# Stop the SparkContext
sc.stop()